In [1]:
#import
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
import matplotlib.pyplot as plt
import datetime
import pandas as pd
import numpy as np
import pymysql
import operator
import os
from scipy import stats
from statsmodels.regression.linear_model import OLS
from datetime import *
import scipy.io as sio
import h5py
import copy
import shutil
import rarfile
import scipy.stats as scs
from general_function import *
from matplotlib_venn import venn3
from matplotlib_venn import venn2
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.font_manager import FontProperties
plt.rcParams['font.sans-serif'] = ['SimHei']  #用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  #用来正常显示负号
sns.set(style="whitegrid")
myfont = FontProperties(fname=r'C:\Windows\Fonts\simhei.ttf', size=10)
sns.set(
    font=myfont.get_name(),
    style="whitegrid",
    palette="muted",
    color_codes=True)

In [2]:
#data
STcode = pd.read_csv('STcode_by_date_for_generator.csv',index_col=0).iloc[-1].dropna().tolist()
code_to_name = pd.read_csv('code_to_name.csv',encoding='gbk',index_col=0)['SEC_NAME']
code_to_name.index = code_to_name.index.str[:6].astype(int)
account_code_to_name = pd.read_excel('account_info.xlsx').set_index('产品标识')['账户备注']
account_code_to_name.index = [i.split(',')[0] if type(i)==str else np.nan for i in account_code_to_name.index]
account_name_to_t0code = pd.read_excel('t0编号.xlsx',index_col=0)['T0代码']

In [3]:
#today
today = datetime.today()
year = str(today.year)
month = today.month
if month < 10:
    month = '0' + str(month)
else:
    month = str(month)
day = today.day
if day < 10:
    day = '0' + str(day)
else:
    day = str(day)    
today_nodot = year + month + day
tomorrow_nodot = get_n_week_date_after(today_nodot)
yesterday_nodot = get_n_week_date_before(today_nodot)

In [4]:
today_nodot,tomorrow_nodot,yesterday_nodot

('20190822', '20190823', '20190821')

today_nodot = '20190815';tomorrow_nodot = '20190816';yesterday_nodot = '20190814'

In [5]:
model_update = '模型分配表.xlsx'  #用的模型记录
total_capital = pd.read_excel('账户资金列表_'+tomorrow_nodot+'.xlsx',index_col=0)
total_model = pd.read_excel(model_update,index_col=1).iloc[:,1:]
sxf = pd.read_excel('分析师买入金额.xlsx',index_col=0)

In [6]:
for account_code in sxf.index.tolist():
    try:
        sub_sxf_capital = sxf.iloc[:,1].xs(account_code)
        total_capital.loc[account_code,'可 用'] = total_capital.loc[account_code,'可 用'] - sub_sxf_capital
        total_capital.loc[account_code,'资 产'] = total_capital.loc[account_code,'资 产'] - 5 * sub_sxf_capital
    except:
        pass

In [7]:
back_checking_days = 12

In [8]:
#理论的总持仓，即如*七天以内（含整七天）*买入标记为H的，*六天以内（含整六天）*买入标记为E的等，买入记录为‘实验_总任务管理表_买入_完成_DATE.xlsx’表格
theoretical_total_holding = pd.DataFrame()
for i in range(0, back_checking_days-1):
    print(get_n_week_date_before(today_nodot,i))
    back_updated_buy = pd.read_excel('实验_总任务管理表_买入_完成_'+get_n_week_date_before(today_nodot,i)+'.xlsx')
    date_model_list = [date_model for date_model in back_updated_buy['模型'].unique().tolist() \
                       if type(date_model) == int and date_model>=int(tomorrow_nodot)]
    total_holding_list = []
    total_holding_list = preset_n_days_model_combined(range(i+2,back_checking_days+1)) + date_model_list
    total_holding_list += list(range(i+2,back_checking_days+1))
    print(total_holding_list)
    sub_theoretical_total_holding = back_updated_buy[back_updated_buy['模型'].isin(total_holding_list)]
    sub_theoretical_total_holding['买入日期'] = get_n_week_date_before(today_nodot,i)
    theoretical_total_holding = pd.concat([theoretical_total_holding,sub_theoretical_total_holding])
theoretical_total_holding['任务资金'] = np.nan
theoretical_total_holding['任务股数'] = theoretical_total_holding['完成股数']
theoretical_total_holding['方向'] = '空'
theoretical_total_holding = theoretical_total_holding.loc[:,['产品标识','股票代码','任务资金','任务股数','方向','模型','买入日期']].reset_index(drop = True)

20190822
['A2', 'A3', 'A4', 'A5', 'B1', 'B3', 'B4', 'C2', 'C3', 'C4', 'C5', 'D1', 'D2', 'K1', 'K2', 'K3', 'K4', 'K5', 'K6', 'K7', 'G1', 'sxf', 'E1', 'E2', 'E3', 'E4', 'F1', 'H1', 'H2', 'H3', 'I1', 'I2', 'I3', 'I4', 'I5', 'J2', 20190902, 20190829, 20190830, 20190823, 20190826, 20190827, 20190828, 20190903, 20190904, 20190905, 20190906, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
20190821
['G1', 'sxf', 'E1', 'E2', 'E3', 'E4', 'F1', 'H1', 'H2', 'H3', 'I1', 'I2', 'I3', 'I4', 'I5', 'J2', 20190830, 20190829, 20190823, 20190826, 20190827, 20190828, 20190902, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
20190820
['sxf', 'E1', 'E2', 'E3', 'E4', 'F1', 'H1', 'H2', 'H3', 'I1', 'I2', 'I3', 'I4', 'I5', 'J2', 20190829, 20190828, 20190823, 4, 5, 6, 7, 8, 9, 10, 11, 12]
20190819
['sxf', 'E1', 'E2', 'E3', 'E4', 'F1', 'H1', 'H2', 'H3', 'I1', 'I2', 'I3', 'I4', 'I5', 'J2', 20190828, 20190827, 20190823, 5, 6, 7, 8, 9, 10, 11, 12]
20190816
['E1', 'E2', 'E3', 'E4', 'F1', 'H1', 'H2', 'H3', 'I1', 'I2', 'I3', 'I4', 'I5', 'J2', 2019

In [9]:
sell_date = []
for i in theoretical_total_holding.index:
    model = theoretical_total_holding.loc[i,'模型']
    buy_date = str(theoretical_total_holding.loc[i,'买入日期'])
    if type(model) == str:
        model = preset_hold_days(model)
        sell_date.append(int(get_n_week_date_after(buy_date,model-1)))
    elif type(model) == int:
        if model > 20000000:
            sell_date.append(model)
        else:
            sell_date.append(int(get_n_week_date_after(buy_date,model-1)))
theoretical_total_holding['卖出日期'] = sell_date
todclose = pd.read_csv('tsdata_nonright_'+today_nodot+'.csv',encoding='gbk',index_col=0).iloc[:,5].replace('<NIL>',np.nan).astype(float)
todclose.index = todclose.index.str[2:].astype(int)
theoretical_total_holding['close'] = todclose.loc[theoretical_total_holding['股票代码']].tolist()
theoretical_total_holding['mkt'] = theoretical_total_holding['close'] * theoretical_total_holding['任务股数']

today_buy = pd.read_excel('实验_总任务管理表_买入_'+today_nodot+'.xlsx')
today_buy['买入日期'] = int(tomorrow_nodot)
sell_date = []
for i in today_buy.index:
    model = today_buy.loc[i,'模型']
    buy_date = str(today_buy.loc[i,'买入日期'])
    if type(model) == str:
        model = 9 if model[0] in 'I' else \
                7 if model[0] in 'H' else \
                6 if model[0] in 'EF' else \
                3 if model[0] in 'G' else \
                2 if model[0] in 'ABCD' else \
                5 if model[0] in 's' else 'wtf'
        sell_date.append(int(get_n_week_date_after(buy_date,model-1)))
    elif type(model) == int:
        if model > 20000000:
            sell_date.append(model)
        else:
            sell_date.append(int(get_n_week_date_after(buy_date,model-1)))
today_buy['卖出日期'] = sell_date
today_buy['mkt'] = today_buy['任务资金']

result = pd.concat([theoretical_total_holding[['产品标识','股票代码','买入日期','卖出日期','模型','mkt']],\
                    today_buy[['产品标识','股票代码','买入日期','卖出日期','模型','mkt']]]).reset_index(drop=True)

In [10]:
result = theoretical_total_holding[['产品标识','股票代码','买入日期','卖出日期','模型','mkt']]
result = result[result['模型']!='sxf']

In [11]:
result = result.set_index(['产品标识','股票代码']).sort_index()
result['mkt_pct'] = result['mkt']/(result.groupby(level=[0,1]).sum()['mkt'])

In [12]:
current_holding = pd.read_excel(today_nodot + '持仓.xlsx')
current_holding['close'] = todclose.loc[current_holding['股票代码']].tolist()
current_holding['mkt'] = current_holding['close'] * current_holding['任务股数']
current_holding = current_holding[current_holding['任务股数']>=100]
current_holding = current_holding[['产品标识','股票代码','mkt']]
current_holding = current_holding.groupby(['产品标识','股票代码']).sum()[['mkt']].sort_index()

In [13]:
result = result.loc[current_holding.index].copy()

In [14]:
result['买入日期'] = result['买入日期'].fillna(today_nodot)
result['卖出日期'] = result['卖出日期'].fillna(int(tomorrow_nodot))
result['模型'] = result['模型'].fillna('2')
result['mkt'] = result['mkt'].fillna(1)
result['mkt_pct'] = result['mkt_pct'].fillna(1)

In [15]:
result['real_mkt'] = result['mkt_pct'] * current_holding.loc[result.index]['mkt']

today_unifinished = pd.read_excel('实验_总任务管理表_买入_未完成_'+today_nodot+'.xlsx', index_col = [0,1])\
.loc[pd.read_excel('买入_未完成_'+today_nodot+'.xlsx', index_col = [0,1]).index]
today_unifinished = today_unifinished[today_unifinished['模型']>int(tomorrow_nodot)]
today_unifinished['买入日期'] = today_nodot
today_unifinished['卖出日期'] = today_unifinished['模型'].astype(int)
today_unifinished['mkt'] = today_unifinished['完成资金']
today_unifinished['mkt_pct'] = 1
today_unifinished['real_mkt'] = today_unifinished['完成资金']
today_unifinished = today_unifinished[['买入日期','卖出日期','模型','mkt','mkt_pct','real_mkt']].reset_index()

In [16]:
result = result.reset_index()
result.columns = ['产品标识','股票代码','买入日期','卖出日期','模型','mkt','mkt_pct','real_mkt']

result = pd.concat([result, today_unifinished]).reset_index(drop = True)

In [17]:
theory_df = []
for account_code in total_capital.index.tolist():
    if account_code in ['HQCC5','HQDB3','HQHF9','HQHF10','HQHT11','HQHZ7','HQYH25','HQYH1','HQYH23','HQYH28','HQGX8','HQYH19','HQCT5','HQDB2']:
        base = 0.96
    elif account_code in ['HQHZ20']:
        base = 0.93
    else:
        base = 1
    current_date = tomorrow_nodot
    date_count = 0
    try:
        sub_model = total_model.xs(account_code)
    except:
        continue
    while(True):
        current_date_weight = 0
        for model_num in range(1,6+1):
            single_model_str = sub_model.xs('第'+str(model_num)+'模型')
            if pd.notnull(single_model_str):
                single_model = single_model_str.split(' * ')[0].replace('(','').replace(')','')
                single_allocation = float(single_model_str.split(' * ')[-1].replace('%',''))/100
                n_hold = preset_hold_days(single_model.split(' ')[0])
                if n_hold > date_count + 1:
                    current_date_weight += base * single_allocation / n_hold  * total_capital.xs(account_code).xs('资 产')
        date_count += 1
        if current_date_weight == 0:
            break
        theory_df.append([account_code, current_date, round(current_date_weight)])
        current_date = get_n_week_date_after(current_date)
theory_df = pd.DataFrame(theory_df)
theory_df.columns = ['产品标识','卖出日期','the_mkt']
theory_df['卖出日期'] = theory_df['卖出日期'].astype(int)
theory_df.set_index(['产品标识','卖出日期'],inplace=True)

In [18]:
final = pd.concat([result.groupby(['产品标识','卖出日期']).sum()[['real_mkt']],theory_df],axis=1).reset_index()

In [19]:
two_days = pd.read_excel('两日模型要买资金.xlsx')
two_days = two_days.groupby('代码').sum()['买入资金']
two_days.index = two_days.index.str.replace('TH','HQ')

In [20]:
final = final.set_index(['产品标识','卖出日期'])
for i in two_days.index:
    try:
        final.loc[(i,int(tomorrow_nodot)),'real_mkt'] += two_days.xs(i)
    except:
        print(i)

In [21]:
final = final.reset_index()
final['diff'] = np.abs(final['real_mkt'] - final['the_mkt'])/final['the_mkt']
final.to_excel('selling_plan_with_theory'+today_nodot+'.xlsx',index = False)